In [2]:
import numpy as np

# --- 1. 量子状態の準備 ---

# 量子ビットの基底ベクトル |0> と |1> を「列ベクトル」として定義 (修正点1)
q0 = np.array([[1], [0]])
q1 = np.array([[0], [1]])

# 複合系の基底ベクトル |00>, |01>, |10>, |11> (これらも列ベクトルになります)
# np.kronはテンソル積を計算する
q00 = np.kron(q0, q0)
q01 = np.kron(q0, q1)
q10 = np.kron(q1, q0)
q11 = np.kron(q1, q1)

# ベル状態 |Φ+> = (|00> + |11>) / sqrt(2)
psi = (q00 + q11) / np.sqrt(2)

# 状態ベクトルから密度行列 ρ_AB = |ψ><ψ| を作成 (修正点2)
# psiが列ベクトル(4,1)になったため、outerではなく行列積を使う
rho_AB = psi @ psi.conj().T

print("--- 複合系の密度行列 ρ_AB (4x4) ---")
# 複素数なので実部だけ表示（この場合は虚部ゼロ）
print(np.real(rho_AB))
print("-" * 30)


# --- 2. 古典的な確率計算（周辺化）---

print("--- 古典的な確率計算 ---")

# 密度行列の対角成分は、各基底状態を見つける確率に対応
# P(A=i, B=j) は <ij|ρ_AB|ij> に対応
P_00 = rho_AB[0, 0] # |00>を見つける確率
P_01 = rho_AB[1, 1] # |01>を見つける確率
P_10 = rho_AB[2, 2] # |10>を見つける確率
P_11 = rho_AB[3, 3] # |11>を見つける確率

print(f"同時確率 P(00): {np.real(P_00):.2f}")
print(f"同時確率 P(01): {np.real(P_01):.2f}")
print(f"同時確率 P(10): {np.real(P_10):.2f}")
print(f"同時確率 P(11): {np.real(P_11):.2f}")
print()

# Aの周辺確率を計算（Bについて和を取る）
# P(A=0) = P(A=0, B=0) + P(A=0, B=1)
P_A0 = P_00 + P_01
# P(A=1) = P(A=1, B=0) + P(A=1, B=1)
P_A1 = P_10 + P_11

print("周辺確率（古典的な方法）:")
print(f"P(A=0) = P(00) + P(01) = {np.real(P_A0):.2f}")
print(f"P(A=1) = P(10) + P(11) = {np.real(P_A1):.2f}")
print("-" * 30)


# --- 3. 量子力学的な計算（部分トレース）---

# q0, q1を列ベクトルにしたことで、ここの計算は意図通りに動くようになります
# np.kron(identity(2,2), q0(2,1)) の結果は (4,2) となる
ket0_B = np.kron(np.identity(2), q0)
ket1_B = np.kron(np.identity(2), q1)

# ket0_Bのshape: (4, 2)
# bra0_Bのshape: (2, 4)
bra0_B = ket0_B.conj().T
bra1_B = ket1_B.conj().T

# ρ_A = <0|_B ρ_AB |0>_B + <1|_B ρ_AB |1>_B
# (2,4) @ (4,4) @ (4,2) の計算となり、次元が合う
rho_A_formal = bra0_B @ rho_AB @ ket0_B + bra1_B @ rho_AB @ ket1_B

print("--- 量子力学的な計算（部分トレース）---")
print("縮約密度行列 ρ_A (2x2):")
print(np.real(rho_A_formal))
print()

# --- 4. 結果の比較 ---

# ρ_A の対角成分が、Aの各状態の確率
P_A0_quantum = rho_A_formal[0, 0]
P_A1_quantum = rho_A_formal[1, 1]

print("周辺確率（部分トレースから）:")
print(f"P(A=0) = (ρ_A)_00 = {np.real(P_A0_quantum):.2f}")
print(f"P(A=1) = (ρ_A)_11 = {np.real(P_A1_quantum):.2f}")
print("-" * 30)

# 比較
print("\n--- 結論 ---")
print(f"古典的手法によるP(A=0): {np.real(P_A0):.2f}, 量子手法によるP(A=0): {np.real(P_A0_quantum):.2f}")
print(f"古典的手法によるP(A=1): {np.real(P_A1):.2f}, 量子手法によるP(A=1): {np.real(P_A1_quantum):.2f}")

if np.isclose(np.real(P_A0), np.real(P_A0_quantum)) and np.isclose(np.real(P_A1), np.real(P_A1_quantum)):
    print("\n結果は一致しました！")
    print("部分トレースは、量子状態における周辺化操作と見なせることが確認できました。")

--- 複合系の密度行列 ρ_AB (4x4) ---
[[0.5 0.  0.  0.5]
 [0.  0.  0.  0. ]
 [0.  0.  0.  0. ]
 [0.5 0.  0.  0.5]]
------------------------------
--- 古典的な確率計算 ---
同時確率 P(00): 0.50
同時確率 P(01): 0.00
同時確率 P(10): 0.00
同時確率 P(11): 0.50

周辺確率（古典的な方法）:
P(A=0) = P(00) + P(01) = 0.50
P(A=1) = P(10) + P(11) = 0.50
------------------------------
--- 量子力学的な計算（部分トレース）---
縮約密度行列 ρ_A (2x2):
[[0.5 0. ]
 [0.  0.5]]

周辺確率（部分トレースから）:
P(A=0) = (ρ_A)_00 = 0.50
P(A=1) = (ρ_A)_11 = 0.50
------------------------------

--- 結論 ---
古典的手法によるP(A=0): 0.50, 量子手法によるP(A=0): 0.50
古典的手法によるP(A=1): 0.50, 量子手法によるP(A=1): 0.50

結果は一致しました！
部分トレースは、量子状態における周辺化操作と見なせることが確認できました。
